In [77]:
import numpy as np
from matplotlib import pyplot as plt
import os
from load_data import load_data
import cv2 
import data_utils
import os

dirname = os.path.dirname(os.path.abspath('Ex1'))
data_path = os.path.join(dirname,'data', 'demo.p')
data = load_data(data_path)

#####Transformation Matrix
num_points = 429

M = np.zeros((4,4,num_points))
homogeneous = np.array([0,0,0,1])
vel = np.zeros((3,num_points))
ang = np.zeros((3,num_points))
#point_cloud = np.zeros((3,num_points))
sample_start = np.zeros(num_points)
sample_camera = np.zeros(num_points)
sample_end = np.zeros(num_points)

time_start = 'data/problem_4/velodyne_points/timestamps_start.txt'
time_camera = 'data/problem_4/velodyne_points/timestamps.txt'
time_end = 'data/problem_4/velodyne_points/timestamps_end.txt'

angle_start_velo = np.zeros(num_points)


point_cloud_p = np.zeros((num_points, point_cloud.shape[0],4))
for i in range(num_points):
    file_index = str(i)
    str_0 = (10-len(file_index))*"0"
    data_path = os.path.join('data/problem_4/oxts/data/', str_0 + file_index+ '.txt')
    vel[:,i] = data_utils.load_oxts_velocity(data_path)
    ang[:,i] = data_utils.load_oxts_angular_rate(data_path)
    sample_start[i] = data_utils.compute_timestamps(time_start, i)
    sample_camera[i] = data_utils.compute_timestamps(time_camera, i)
    sample_end[i] = data_utils.compute_timestamps(time_end, i)
    #3. Compute translation using angular velocity and velocity
    pos_xy_cam = -vel[0:2, i]*(sample_camera[i]-sample_start[i])
    #4.Compute rotation using pos and angular velocity
    teta = -ang[2, i]*(sample_camera[i]-sample_start[i])
    R = np.array([[np.cos(teta), -np.sin(teta), 0],
                [np.sin(teta), np.cos(teta), 0],
                [0,             0,     1]])
    T = np.array([[pos_xy_cam[0]],
                [pos_xy_cam[1]],
                [0]])

    inter = np.hstack((R,T))
    inter = np.vstack((inter, homogeneous))
    M[:,:,i] = inter
    
    w_velodyne = 2*np.pi/(sample_end[i]-sample_start[i])
    ang_no_corr = w_velodyne*(sample_camera[i]-sample_start[i])
    angle_start_velo[i] = ang_no_corr + teta
         
    ####Velodyne points
    file_index = str(i)
    str_0 = (10-len(file_index))*"0"
    data_velodyne = os.path.join('data/problem_4/velodyne_points/data/', str_0 + file_index+ '.bin')
    point_cloud = data_utils.load_from_bin(data_velodyne)
    alpha = np.zeros(point_cloud.shape[0])
    for j in range(point_cloud.shape[0]):
        hyp = np.sqrt(point_cloud[j,0]**2+point_cloud[j,1]**2)
        alpha[j] = np.arcsin(point_cloud[j,1]/hyp)
        cos = point_cloud[j,0]/hyp
        if cos < 0:
            alpha[j] = alpha[j] + np.pi
        if alpha[j] < 0:
            alpha[j] = 2*np.pi + alpha[j]
        #alpha[j] = alpha[j] - angle_start_velo[i]
    indices = np.argsort(alpha)
    alpha = np.sort(alpha) - angle_start_velo[i]
    
    start_point = int(min(np.argwhere(alpha >= 0)))
    alpha_1 = alpha[start_point:]
    alpha_2 = alpha[0:start_point]
    alpha = np.append(alpha_1,alpha_2)
    
    
    ind_start_point = int(np.argwhere(indices == start_point))
    indices_1 = indices[ind_start_point:]
    indices_2 = indices[0:ind_start_point]
    indices = np.append(indices_1 ,indices_2)
    
    ##point cloud velodyne sorted in scan order
    point_cloud = point_cloud[indices,:]
    
    ##rectification matrices for each point
    #delta time between each point in the scan
    delta_t = (sample_start[i]-sample_end[i])/point_cloud.shape[0]
    
    M_p = np.zeros((4,4,point_cloud.shape[0]))
    for j in range(point_cloud.shape[0]):
        #Compute translation using angular velocity and velocity
        pos_xy_p = -vel[0:2, i]*j*delta_t
        #Compute rotation using pos and angular velocity
        teta_p = -ang[2, i]*j*delta_t
        R = np.array([[np.cos(teta_p), -np.sin(teta_p), 0],
                    [np.sin(teta_p), np.cos(teta_p), 0],
                    [0,             0,     1]])
        T = np.array([[pos_xy_p[0]],
                    [pos_xy_p[1]],
                    [0]])

        inter = np.hstack((R,T))
        inter = np.vstack((inter, homogeneous))
        M_p[:,:,j] = inter
        
        intr= np.hstack((point_cloud[j,:], 1))
        point_cloud_p[i,j,:]  = intr
        point_cloud_p[i,j,:] = np.matmul(point_cloud_p[i,j,:], M_p[:,:,j])
   
        

[-2.365  3.533 -1.774]


KeyboardInterrupt: 